In [2]:
import pandas as pd
import numpy as np

In [3]:
def round_hour(row, variable):
    if int(row[variable + '_min']) > 30:
        return int(row[variable +'_hour']) + 1
    else:
        return int(row[variable +'_hour'])

In [10]:
def get_cyclical_time(df, variable):
    df[variable]= df[variable].replace([2400],0)
    df[variable] = df[variable].astype(int).astype(str).str.zfill(4) 
    df[variable] = pd.to_datetime(df[variable], format='%H%M')
    df[variable +'_hour'] = df[variable].dt.hour
    df[variable + '_min'] = df[variable].dt.minute
    df[variable +'_hour'] = df.apply(lambda x: round_hour(x, variable), axis=1)
    df['sin_' + variable] = np.sin(df[variable +'_hour'])
    df['cos_' + variable] = np.cos(df[variable +'_hour'])

In [5]:
def is_delayed(row):
    if row['arr_delay'] <= 15:
        return 0
    else:
        return 1

def how_delayed(row):
    if row['arr_delay'] <= 15:
        return 0
    else:
        return row['arr_delay']

In [9]:
def get_features(path, path2, tvt, csv):
    df = pd.read_csv(path + csv)
    dftest = pd.read_csv(path2 + csv)
    df['fl_date'] = pd.to_datetime(df['fl_date'])
    df['date']= df['fl_date'].dt.day
    df['weekday'] = df['fl_date'].dt.dayofweek
    df['sin_weekday'] = np.sin(df['weekday'])
    df['cos_weekday'] = np.cos(df['weekday'])
    dftest['fl_date'] = pd.to_datetime(dftest['fl_date'])
    dftest['date']= dftest['fl_date'].dt.day
    dftest['weekday'] = dftest['fl_date'].dt.dayofweek
    dftest['sin_weekday'] = np.sin(dftest['weekday'])
    dftest['cos_weekday'] = np.cos(dftest['weekday'])
    get_cyclical_time(df, 'crs_dep_time')
    get_cyclical_time(dftest, 'crs_dep_time')
    get_cyclical_time(df, 'dep_time')
    get_cyclical_time(df, 'crs_arr_time')
    get_cyclical_time(dftest, 'crs_arr_time')
    get_cyclical_time(df, 'arr_time')
    df['delayed'] = df.apply(lambda x: is_delayed(x), axis=1)
    df['delay'] = df.apply(lambda x: how_delayed(x), axis=1)
    # average departure delay and number of fligths per carrier
    df = df.join(df.groupby('op_unique_carrier').mean()['dep_delay'], on='op_unique_carrier', rsuffix='A').rename({'dep_delayA': 'DelayD_carrier'}, axis=1)
    df = df.join(df.groupby('op_unique_carrier').count()['fl_date'], on='op_unique_carrier', rsuffix='A').rename({'fl_dateA': 'num_flights_carrier'}, axis=1)
    dftest = dftest.join(df.groupby('op_unique_carrier').mean()['dep_delay'], on='op_unique_carrier', rsuffix='A').rename({'dep_delayA': 'DelayD_carrier'}, axis=1)
    dftest = dftest.join(dftest.groupby('op_unique_carrier').count()['fl_date'], on='op_unique_carrier', rsuffix='A').rename({'fl_dateA': 'num_flights_carrier'}, axis=1)
    # average departure delay and number of flights per origin
    df = df.join(df.groupby('origin').mean()['dep_delay'], on='origin', rsuffix='A').rename({'dep_delayA': 'DelayD_origin'}, axis=1)
    df = df.join(df.groupby('origin').count()['fl_date'], on='origin', rsuffix='A').rename({'fl_dateA': 'num_flights_origin'}, axis=1)
    dftest = dftest.join(df.groupby('origin').mean()['dep_delay'], on='origin', rsuffix='A').rename({'dep_delayA': 'DelayD_origin'}, axis=1)
    dftest = dftest.join(dftest.groupby('origin').count()['fl_date'], on='origin', rsuffix='A').rename({'fl_dateA': 'num_flights_origin'}, axis=1)
    # average departure delay and number of flights per destination
    df = df.join(df.groupby('dest').mean()['dep_delay'], on='dest', rsuffix='A').rename({'dep_delayA': 'DelayD_dest'}, axis=1)
    df = df.join(df.groupby('dest').count()['fl_date'], on='dest', rsuffix='A').rename({'fl_dateA': 'num_flights_dest'}, axis=1)
    dftest = dftest.join(df.groupby('dest').mean()['dep_delay'], on='dest', rsuffix='A').rename({'dep_delayA': 'DelayD_dest'}, axis=1)
    dftest = dftest.join(dftest.groupby('dest').count()['fl_date'], on='dest', rsuffix='A').rename({'fl_dateA': 'num_flights_dest'}, axis=1)
    # average arrival delay per carrier, origin and destination
    df = df.join(df.groupby('op_unique_carrier').mean()['arr_delay'], on='op_unique_carrier', rsuffix='A').rename({'arr_delayA': 'DelayA_carrier'}, axis=1)
    df = df.join(df.groupby('origin').mean()['arr_delay'], on='origin', rsuffix='A').rename({'arr_delayA': 'DelayA_origin'}, axis=1)
    df = df.join(df.groupby('dest').mean()['arr_delay'], on='dest', rsuffix='A').rename({'arr_delayA': 'DelayA_dest'}, axis=1)
    dftest = dftest.join(df.groupby('op_unique_carrier').mean()['arr_delay'], on='op_unique_carrier', rsuffix='A').rename({'arr_delayA': 'DelayA_carrier'}, axis=1)
    dftest = dftest.join(df.groupby('origin').mean()['arr_delay'], on='origin', rsuffix='A').rename({'arr_delayA': 'DelayA_origin'}, axis=1)
    dftest = dftest.join(df.groupby('dest').mean()['arr_delay'], on='dest', rsuffix='A').rename({'arr_delayA': 'DelayA_dest'}, axis=1)
    # average arrival delay and number of flights per hour(Dep)
    df = df.join(df.groupby('crs_dep_time_hour').mean()['arr_delay'], on='crs_dep_time_hour', rsuffix='A').rename({'arr_delayA': 'DelayA_hourly'}, axis=1)
    df = df.join(df.groupby('crs_dep_time_hour').count()['arr_delay'], on='crs_dep_time_hour', rsuffix='A').rename({'arr_delayA': 'num_flights_hourly'}, axis=1)
    dftest = dftest.join(df.groupby('crs_dep_time_hour').mean()['arr_delay'], on='crs_dep_time_hour', rsuffix='A').rename({'arr_delayA': 'DelayA_hourly'}, axis=1)
    dftest = dftest.join(dftest.groupby('crs_dep_time_hour').count()['fl_date'], on='crs_dep_time_hour', rsuffix='A').rename({'fl_dateA': 'num_flights_hourly'}, axis=1)
    # average departure delay per hour(Dep)
    df = df.join(df.groupby('crs_dep_time_hour').mean()['dep_delay'], on='crs_dep_time_hour', rsuffix='A').rename({'dep_delayA': 'DelayD_hourly'}, axis=1)
    dftest = dftest.join(df.groupby('crs_dep_time_hour').mean()['dep_delay'], on='crs_dep_time_hour', rsuffix='A').rename({'dep_delayA': 'DelayD_hourly'}, axis=1)
    # average arrival delay per hour (per origin and per carrier)
    df = df.join(df.groupby(['crs_dep_time_hour', 'origin']).mean()['arr_delay'], on=['crs_dep_time_hour', 'origin'], rsuffix='A').rename({'arr_delayA': 'DelayA_origin_hourly'}, axis=1)
    df = df.join(df.groupby(['crs_dep_time_hour', 'op_unique_carrier']).mean()['arr_delay'], on=['crs_dep_time_hour', 'op_unique_carrier'], rsuffix='A').rename({'arr_delayA': 'DelayA_carrier_hourly'}, axis=1)
    dftest = dftest.join(df.groupby(['crs_dep_time_hour', 'origin']).mean()['arr_delay'], on=['crs_dep_time_hour', 'origin'], rsuffix='A').rename({'arr_delayA': 'DelayA_origin_hourly'}, axis=1)
    dftest = dftest.join(df.groupby(['crs_dep_time_hour', 'op_unique_carrier']).mean()['arr_delay'], on=['crs_dep_time_hour', 'op_unique_carrier'], rsuffix='A').rename({'arr_delayA': 'DelayA_carrier_hourly'}, axis=1)
    # average arrival delay per hour (per origin and per carrier)
    df = df.join(df.groupby(['crs_dep_time_hour', 'origin']).mean()['dep_delay'], on=['crs_dep_time_hour', 'origin'], rsuffix='A').rename({'dep_delayA': 'DelayD_origin_hourly'}, axis=1)
    df = df.join(df.groupby(['crs_dep_time_hour', 'op_unique_carrier']).mean()['dep_delay'], on=['crs_dep_time_hour', 'op_unique_carrier'], rsuffix='A').rename({'dep_delayA': 'DelayD_carrier_hourly'}, axis=1)
    dftest = dftest.join(df.groupby(['crs_dep_time_hour', 'origin']).mean()['dep_delay'], on=['crs_dep_time_hour', 'origin'], rsuffix='A').rename({'dep_delayA': 'DelayD_origin_hourly'}, axis=1)
    dftest = dftest.join(df.groupby(['crs_dep_time_hour', 'op_unique_carrier']).mean()['dep_delay'], on=['crs_dep_time_hour', 'op_unique_carrier'], rsuffix='A').rename({'dep_delayA': 'DelayD_carrier_hourly'}, axis=1)
    df.to_csv(f'{path}engineered_{csv}', index=None)
    dftest.to_csv(f'{path}engineered_test{csv}', index=None)

In [12]:
path = 'Training/'
train = 'train_'
test = 'test_'
file1 = 'finalcleaned_finalraw_flights_Dec_Jan.csv'
file2 = 'finalcleaned_finalraw_flights_Dec.csv' 
file3 = 'finalcleaned_finalraw_flights_Jan_1w.csv' 
file4 = 'finalcleaned_finalraw_flights_Jan_2w.csv' 
file5 = 'finalcleaned_finalraw_flights_Jan.csv' 
files = [file1,file2,file3,file4,file5]

In [11]:
for i in files:
    get_features(path, train, i)

TypeError: get_features() missing 1 required positional argument: 'csv'

In [15]:
path2 = 'Validation/'
for i in files:
    get_features(path, test, i)

In [13]:
path2 = 'Validation/'
get_features(path, path2, test, file3)

FileNotFoundError: [Errno 2] No such file or directory: 'Training/finalcleaned_finalraw_flights_Jan_1w.csv'